# Práctica: Apache Kafka & PySpark

El objetivo de esta práctica es construir una aplicación que permite la ingesta de tweets usando Apache Kafka y puedan ser procesados como flujos de datos estructurados [Structured Streaming](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html).

![my_test_image](https://turing.iimas.unam.mx/~blancavg/images/app_twitter.png)

### ¿Qué es Apache Kafka?

Kafka es una plataforma distribuida de transmisión de datos que permite publicar, almacenar, y procesar en tiempo real. Kafka es un proyecto de código abierto desarrollado por LinkedIn y donado a la Apache Software Foundation escrito en Java y Scala.

Ventajas que ofrece Kafka en comparación a otras opciones para el manejo de datos más tradicionales:

- Garantiza un procesamiento rápido y fluido de datos.
- Escalabilidad
- Tolerancia a fallos
- Pueden manejar escritura y lectura de datos de alta frecuencia
- Unifica los canales de datos entre microservicios (contenedores).

#### Conceptos

- Topico: Un tópico es un flujo de datos, los cuales se guardan en forma formato (llave, valor). Cada evento (dato) que llega al sistema debe ser parte de un tópico. 

- Productores: Son las apps o microservicios que  publican datos en el sistema de Kafka. Publican datos en los tópicos de su eleccion.

- Consumidores: Son las apps o microservicios que usan los datos publicados por los productores. Para consumir datos, un consumidor debe suscribirse a un tópico.

- Broker: es el conjunto de servidores (clúster) donde se ejecutar Kafka. Los datos de un tópico son replicados y particionados en varios brokers, los cual permite a los consumidores leer datos en paralelo y tolerar fallos.

### Instalación de Apache Kafka

En esta sección se describe un conjunto de pasos necesarios para la instalación y funcionamiento de Apache Kafka.

In [0]:
%sh hostname

0503-121551-7gk1ix33-10-172-192-246


In [0]:
%sh cat /etc/os-release

NAME="Ubuntu"
VERSION="20.04.4 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.4 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


In [0]:
%sh java -version

openjdk version "1.8.0_302"
OpenJDK Runtime Environment (Zulu 8.56.0.21-CA-linux64) (build 1.8.0_302-b08)
OpenJDK 64-Bit Server VM (Zulu 8.56.0.21-CA-linux64) (build 25.302-b08, mixed mode)


In [0]:
%sh whoami

root


##### Descarga de Apache Kafka

In [0]:
%sh wget https://dlcdn.apache.org/kafka/3.1.0/kafka_2.12-3.1.0.tgz

--2022-05-03 12:38:35--  https://dlcdn.apache.org/kafka/3.1.0/kafka_2.12-3.1.0.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88217241 (84M) [application/x-gzip]
Saving to: ‘kafka_2.12-3.1.0.tgz’

     0K .......... .......... .......... .......... ..........  0% 5.39M 16s
    50K .......... .......... .......... .......... ..........  0% 6.60M 14s
   100K .......... .......... .......... .......... ..........  0% 5.10M 15s
   150K .......... .......... .......... .......... ..........  0% 8.85M 14s
   200K .......... .......... .......... .......... ..........  0% 6.36M 13s
   250K .......... .......... .......... .......... ..........  0% 5.14M 14s
   300K .......... .......... .......... .......... ..........  0% 8.53M 13s
   350K .......... .......... .......... .......... ..........  0% 5.93M 13s
   400K .....

In [0]:
%sh ls -ltr ./

total 87452
-r-xr-xr-x 1 root root  1307454 Jan  1  1970 preload_class.lst
drwxr-xr-x 2 root root     4096 Jan  1  1970 conf
-rw-r--r-- 1 root root 88217241 Jan 21 14:54 kafka_2.12-3.1.0.tgz
drwxr-xr-x 3 root root     4096 May  3 12:19 eventlogs
drwxr-xr-x 2 root root     4096 May  3 12:20 logs
drwxr-xr-x 5 root root     4096 May  3 12:22 metastore_db
drwxr-xr-x 2 root root     4096 May  3 12:30 ganglia


##### Extracción de Apache Kafka

In [0]:
%sh tar -xzf kafka_2.12-3.1.0.tgz

In [0]:
%sh ls -ltr ./

total 87456
-r-xr-xr-x 1 root root  1307454 Jan  1  1970 preload_class.lst
drwxr-xr-x 2 root root     4096 Jan  1  1970 conf
drwxr-xr-x 7 root root     4096 Jan 12 09:04 kafka_2.12-3.1.0
-rw-r--r-- 1 root root 88217241 Jan 21 14:54 kafka_2.12-3.1.0.tgz
drwxr-xr-x 3 root root     4096 May  3 12:19 eventlogs
drwxr-xr-x 2 root root     4096 May  3 12:20 logs
drwxr-xr-x 5 root root     4096 May  3 12:22 metastore_db
drwxr-xr-x 2 root root     4096 May  3 12:30 ganglia


##### Inicio del servicio de ZooKeeper

ZooKeeper es una solución de consistencia de datos distribuidos, dedicada a proporcionar un servicio de coordinación distribuida con alto rendimiento, alta disponibilidad y estrictas capacidades de control de acceso secuencial para aplicaciones distribuidas.

ZooKeeper es un servicio que emplea Kafka para administrar los brokers.

In [0]:
%sh cd kafka_2.12-3.1.0
ls -ltr ./
bin/zookeeper-server-start.sh config/zookeeper.properties

total 64
-rw-r--r-- 1 root root 28184 Jan 12 09:01 NOTICE
-rw-r--r-- 1 root root 14536 Jan 12 09:01 LICENSE
drwxr-xr-x 2 root root  4096 Jan 12 09:04 licenses
drwxr-xr-x 3 root root  4096 Jan 12 09:04 config
drwxr-xr-x 3 root root  4096 Jan 12 09:04 bin
drwxr-xr-x 2 root root  4096 Jan 12 09:04 site-docs
drwxr-xr-x 2 root root  4096 May  3 12:38 libs
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/databricks/jars/----workspace_spark_3_2--maven-trees--hive-2.3__hadoop-3.2--org.slf4j--slf4j-log4j12--org.slf4j__slf4j-log4j12__1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/databricks/driver/kafka_2.12-3.1.0/libs/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
[2022-05-03 12:38:51,489] INFO Reading configuration from: config/zookeeper.properties 